In [5]:
# SOURCE: https://github.com/samchaaa/EDGAR

import requests, json, re, xmltodict
import pandas as pd
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

### To do ###

# Add basic email
# Add up derivatives
# Add in time handling (take from other PA things)


In [6]:
baseUrl = 'https://www.sec.gov/'
# sp500 = list(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies', header=0)[0]['Symbol'].values)

In [7]:
def getEDGAR(pages):
    form4 = []
    for p in range(pages): # can go out to 2001-2100 (20 pages)   
        url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcurrent&datea=&dateb=&company=&type=&SIC=&State=&Country=&CIK=&owner=include&accno=&start={}&count=100'.format(p*100)
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        form4.extend(
            [
                {
                    'link': tr.find('a', text=re.compile('html'))['href'], 
                    'time': tr.find_all('td')[3].text
                } 
                for tr in soup.find_all('tr') if tr.find('td', text=re.compile('^4$'))
            ]
        )
    return form4

def getForm(edgarUrl):
    r = requests.get(baseUrl + edgarUrl)
    soup = BeautifulSoup(r.text, 'html.parser')
    for tr in soup.find_all('tr'):
        if tr.find('td', text=re.compile('^4$')):
            if tr.find('a', text=re.compile('html')):
                html = tr.find('a', text=re.compile('html'))['href']
            if tr.find('a', text=re.compile('xml')):
                xml = tr.find('a', text=re.compile('xml'))['href']
    return {'html': html, 'xml': xml}

def getAllData(xmlUrl):
    r = requests.get(baseUrl + xmlUrl)
    soup = BeautifulSoup(r.text, 'html.parser')
    data = xmltodict.parse(soup.prettify())
    data = json.loads(json.dumps(data))
    data = data['ownershipdocument']
    return data

# jfc thank you so much for this article: https://towardsdatascience.com/how-to-flatten-deeply-nested-json-objects-in-non-recursive-elegant-python-55f96533103d

def flatten_json(nested_json):
    out = {}
    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x
    flatten(nested_json)
    return out

def getLinks(mainLinksList):
    newLinksList = []
    for x in mainLinksList:
        form = getForm(x['link'])
        form['time'] = x['time']
        newLinksList.append(form)
    return newLinksList

In [8]:
pull = getEDGAR(1)
data = getLinks(pull[:])

In [9]:
results = []
for x in data[:]:
    f4 = getAllData(x['xml'])
    #f4 = flatten_json(f4)
    f4['timestamp'] = x['time']
    f4['xml'] = x['xml']
    f4['html'] = x['html']
    results.append(f4)
#results = pd.DataFrame(results)

In [10]:
pd.DataFrame(results) #.dropna(thresh=10, how='all', axis=1)

,derivativetable,documenttype,footnotes,html,issuer,nonderivativetable,notsubjecttosection16,ownersignature,periodofreport,remarks,reportingowner,schemaversion,timestamp,xml
0,NaN,4,"{'footnote': {'@id': 'F1', '#text': 'Represent...",/Archives/edgar/data/1037868/00011276021903238...,"{'issuercik': '0001037868', 'issuername': 'AME...",{'nonderivativetransaction': [{'securitytitle'...,NaN,"{'signaturename': '/s/ Lynn Carino, attorney-i...",2019-10-21,NaN,{'reportingownerid': {'rptownercik': '00015496...,X0306,2019-11-0821:41:18,/Archives/edgar/data/1037868/00011276021903238...
1,NaN,4,"{'footnote': {'@id': 'F1', '#text': 'Represent...",/Archives/edgar/data/1037868/00011276021903238...,"{'issuercik': '0001037868', 'issuername': 'AME...",{'nonderivativetransaction': [{'securitytitle'...,NaN,"{'signaturename': '/s/ Lynn Carino, attorney-i...",2019-10-21,NaN,{'reportingownerid': {'rptownercik': '00015496...,X0306,2019-11-0821:41:18,/Archives/edgar/data/1037868/00011276021903238...
2,{'derivativetransaction': {'securitytitle': {'...,4,"{'footnote': {'@id': 'F1', '#text': 'The stock...",/Archives/edgar/data/1296445/00014377491902223...,"{'issuercik': '0001296445', 'issuername': 'ORM...",NaN,NaN,{'signaturename': '/s/ Etty Rosner- as attorne...,2019-11-07,NaN,{'reportingownerid': {'rptownercik': '00017117...,X0306,2019-11-0821:38:45,/Archives/edgar/data/1296445/00014377491902223...
3,{'derivativetransaction': {'securitytitle': {'...,4,"{'footnote': {'@id': 'F1', '#text': 'The stock...",/Archives/edgar/data/1296445/00014377491902223...,"{'issuercik': '0001296445', 'issuername': 'ORM...",NaN,NaN,{'signaturename': '/s/ Etty Rosner- as attorne...,2019-11-07,NaN,{'reportingownerid': {'rptownercik': '00017117...,X0306,2019-11-0821:38:45,/Archives/edgar/data/1296445/00014377491902223...
4,{'derivativetransaction': {'securitytitle': {'...,4,"{'footnote': {'@id': 'F1', '#text': 'Each rest...",/Archives/edgar/data/1296445/00014377491902223...,"{'issuercik': '0001296445', 'issuername': 'ORM...",{'nonderivativetransaction': {'securitytitle':...,NaN,{'signaturename': '/s/ Etty Rosner- as attorne...,2019-11-07,NaN,{'reportingownerid': {'rptownercik': '00016312...,X0306,2019-11-0821:38:13,/Archives/edgar/data/1296445/00014377491902223...
5,{'derivativetransaction': {'securitytitle': {'...,4,"{'footnote': {'@id': 'F1', '#text': 'Each rest...",/Archives/edgar/data/1296445/00014377491902223...,"{'issuercik': '0001296445', 'issuername': 'ORM...",{'nonderivativetransaction': {'securitytitle':...,NaN,{'signaturename': '/s/ Etty Rosner- as attorne...,2019-11-07,NaN,{'reportingownerid': {'rptownercik': '00016312...,X0306,2019-11-0821:38:13,/Archives/edgar/data/1296445/00014377491902223...
6,{'derivativetransaction': {'securitytitle': {'...,4,"{'footnote': {'@id': 'F1', '#text': 'The stock...",/Archives/edgar/data/1296445/00014377491902223...,"{'issuercik': '0001296445', 'issuername': 'ORM...",NaN,NaN,{'signaturename': '/s/ Etty Rosner- as attorne...,2019-11-07,NaN,{'reportingownerid': {'rptownercik': '00017112...,X0306,2019-11-0821:37:41,/Archives/edgar/data/1296445/00014377491902223...
7,{'derivativetransaction': {'securitytitle': {'...,4,"{'footnote': {'@id': 'F1', '#text': 'The stock...",/Archives/edgar/data/1296445/00014377491902223...,"{'issuercik': '0001296445', 'issuername': 'ORM...",NaN,NaN,{'signaturename': '/s/ Etty Rosner- as attorne...,2019-11-07,NaN,{'reportingownerid': {'rptownercik': '00017112...,X0306,2019-11-0821:37:41,/Archives/edgar/data/1296445/00014377491902223...
8,{'derivativetransaction': [{'securitytitle': {...,4,"{'footnote': [{'@id': 'F1', '#text': 'Each res...",/Archives/edgar/data/1181711/00014377491902222...,"{'issuercik': '0001296445', 'issuername': 'ORM...",{'nonderivativetransaction': {'securitytitle':...,NaN,{'signaturename': '/s/ Etty Rosner- as attorne...,2019-11-07,NaN,{'reportingownerid': {'rptownercik': '00011817...,X0306,2019-11-0821:37:09,/Archives/edgar/data/1181711/00014377491902222...
9,{'derivativetransaction': [{

In [11]:
def cleanTx(x):
    
    newX = {}
    
    if 'derivativetable' in x.keys():
        try:
            if type(x['derivativetable']['derivativetransaction']) == list:
                newX['devs'] = len(x['derivativetable']['derivativetransaction'])
                newX['devsTypes'] = list(set([y['securitytitle']['value'] for y in x['derivativetable']['derivativetransaction']]))     
            else:
                newX['devs'] = 1
                newX['devsTypes'] = x['derivativetable']['derivativetransaction']['securitytitle']['value'] 
        except:
            newX['devs'] = 'error'
            newX['devsTypes'] = 'error'
    if 'nonderivativetable' in x.keys():
        try:
            if type(x['nonderivativetable']['nonderivativetransaction']) == list:
                newX['nonDev'] = len(x['nonderivativetable']['nonderivativetransaction'])
                newX['nonDevTypes'] = list(set([y['securitytitle']['value'] for y in x['nonderivativetable']['nonderivativetransaction']]))     
            else:
                newX['nonDev'] = 1
                newX['nonDevTypes'] = x['nonderivativetable']['nonderivativetransaction']['securitytitle']['value']  
        except:
            newX['nonDev'] = 'error'
            newX['nonDevTypes'] = 'error'
         
    newX['issuer'] = x['issuer']['issuername']
    newX['sym'] = x['issuer']['issuertradingsymbol']
    #newX['reporting'] = x['reportingowner']['reportingownerid']['rptownername']
    #newX['relation'] = x['reportingowner']['reportingownerrelationship']
    
    return newX

In [20]:
results[0]

{'schemaversion': 'X0306',
 'documenttype': '4',
 'periodofreport': '2019-10-21',
 'issuer': {'issuercik': '0001037868',
  'issuername': 'AMETEK INC/',
  'issuertradingsymbol': 'AME'},
 'reportingowner': {'reportingownerid': {'rptownercik': '0001549620',
   'rptownername': 'Burke William Joseph'},
  'reportingowneraddress': {'rptownerstreet1': '1100 CASSATT ROAD',
   'rptownerstreet2': None,
   'rptownercity': 'BERWYN',
   'rptownerstate': 'PA',
   'rptownerzipcode': '19312',
   'rptownerstatedescription': None},
  'reportingownerrelationship': {'isofficer': '1',
   'officertitle': 'Executive VP - CFO'}},
 'nonderivativetable': {'nonderivativetransaction': [{'securitytitle': {'value': 'Common Stock/ Serp'},
    'transactiondate': {'value': '2019-11-08'},
    'transactioncoding': {'transactionformtype': '4',
     'transactioncode': 'J',
     'equityswapinvolved': '0',
     'footnoteid': {'@id': 'F1'}},
    'transactiontimeliness': None,
    'transactionamounts': {'transactionshares': {'

In [19]:
cleanTx(results[0])

{'nonDev': 4,
 'nonDevTypes': ['Common Stock/ Serp', 'Common Stock'],
 'issuer': 'AMETEK INC/',
 'sym': 'AME'}

In [12]:
pd.DataFrame([cleanTx(x) for x in results]).fillna('')

,devs,devsTypes,issuer,nonDev,nonDevTypes,sym
0,,,AMETEK INC/,4,"[Common Stock/ Serp, Common Stock]",AME
1,,,AMETEK INC/,4,"[Common Stock/ Serp, Common Stock]",AME
2,1,Stock Appreciation Right(SAR),"ORMAT TECHNOLOGIES, INC.",,,ORA
3,1,Stock Appreciation Right(SAR),"ORMAT TECHNOLOGIES, INC.",,,ORA
4,1,Restricted Stock Units,"ORMAT TECHNOLOGIES, INC.",1,Common Stock,ORA
5,1,Restricted Stock Units,"ORMAT TECHNOLOGIES, INC.",1,Common Stock,ORA
6,1,Stock Appreciation Right(SAR),"ORMAT TECHNOLOGIES, INC.",,,ORA
7,1,Stock Appreciation Right(SAR),"ORMAT TECHNOLOGIES, INC.",,,ORA
8,3,"[Stock Appreciation right, Restricted Stock Un...","ORMAT TECHNOLOGIES, INC.",1,Common Stock,ORA
9,3,"[Stock Appreciation right, Restricted Stock Un...","ORMAT TECHNOLOGIES, INC.",1,Common Stock,ORA


In [767]:
results

[{'schemaversion': 'X0306',
  'documenttype': '4',
  'periodofreport': '2019-10-21',
  'issuer': {'issuercik': '0001037868',
   'issuername': 'AMETEK INC/',
   'issuertradingsymbol': 'AME'},
  'reportingowner': {'reportingownerid': {'rptownercik': '0001549620',
    'rptownername': 'Burke William Joseph'},
   'reportingowneraddress': {'rptownerstreet1': '1100 CASSATT ROAD',
    'rptownerstreet2': None,
    'rptownercity': 'BERWYN',
    'rptownerstate': 'PA',
    'rptownerzipcode': '19312',
    'rptownerstatedescription': None},
   'reportingownerrelationship': {'isofficer': '1',
    'officertitle': 'Executive VP - CFO'}},
  'nonderivativetable': {'nonderivativetransaction': [{'securitytitle': {'value': 'Common Stock/ Serp'},
     'transactiondate': {'value': '2019-11-08'},
     'transactioncoding': {'transactionformtype': '4',
      'transactioncode': 'J',
      'equityswapinvolved': '0',
      'footnoteid': {'@id': 'F1'}},
     'transactiontimeliness': None,
     'transactionamounts': 

In [768]:
# cleanData(results)

In [582]:
# results.columns

In [501]:
# results.isna().sum()

In [596]:
results.dropna(thresh=1, how='all', axis=1).T

,0,1,2,3,4
derivativetable_derivativetransaction_conversionorexerciseprice_footnoteid_@id,F1,F1,F1,F1,F1
derivativetable_derivativetransaction_exercisedate_footnoteid_@id,F3,F3,F2,F2,F3
derivativetable_derivativetransaction_expirationdate_value,2026-05-07,2026-05-07,2026-05-07,2026-05-07,2026-05-07
derivativetable_derivativetransaction_ownershipnature_directorindirectownership_value,D,D,D,D,D
derivativetable_derivativetransaction_posttransactionamounts_sharesownedfollowingtransaction_value,16420,16420,16420,16420,16420
derivativetable_derivativetransaction_securitytitle_value,Restricted Stock Unit,Restricted Stock Unit,Restricted Stock Unit,Restricted Stock Unit,Restricted Stock Unit
derivativetable_derivativetransaction_transactionamounts_transactionacquireddisposedcode_value,D,D,D,D,D
derivativetable_derivativetransaction_transactionamounts_transactionpricepershare_value,0.00,0.00,0.00,0.00,0.00
derivativetable_derivativetransaction_transactionamounts_transactionshares_value,8210,8210,8210,8210,8210
derivativetable_derivativetransaction_transactioncoding_equityswapinvolved,0,0,0,0,0


In [597]:
results.loc[0].dropna()

derivativetable_derivativetransaction_conversionorexerciseprice_footnoteid_@id                                                                             F1
derivativetable_derivativetransaction_exercisedate_footnoteid_@id                                                                                          F3
derivativetable_derivativetransaction_expirationdate_value                                                                                         2026-05-07
derivativetable_derivativetransaction_ownershipnature_directorindirectownership_value                                                                       D
derivativetable_derivativetransaction_posttransactionamounts_sharesownedfollowingtransaction_value                                                      16420
derivativetable_derivativetransaction_securitytitle_value                                                                               Restricted Stock Unit
derivativetable_derivativetransaction_transactionamo

In [14]:
# # What to keep

# """

# 'html'
# 'timestamp'
# 'issuer_issuertradingsymbol'
# 'issuer_issuername'
# 'periodofreport'
# 'reportingowner_reportingownerrelationship_officertitle'
# 'nonderivativetable_nonderivativetransaction_0_securitytitle_value'
# 'nonderivativetable_nonderivativetransaction_0_posttransactionamounts_sharesownedfollowingtransaction_value'
# 'nonderivativetable_nonderivativetransaction_0_transactionamounts_transactionacquireddisposedcode_value'
# 'nonderivativetable_nonderivativetransaction_0_transactionamounts_transactionshares_value'
# 'reportingowner_reportingownerid_rptownername'
# > Do they own derivatives? Yes or no.
# 'nonderivativetable_nonderivativetransaction_0_transactioncoding_transactioncode'                                                                           S
# 'nonderivativetable_nonderivativetransaction_0_transactioncoding_transactionformtype'                                                              4
# 'nonderivativetable_nonderivativetransaction_0_transactiondate_value'

# --- Form 4 Transaction Codes ---

# General Transaction Codes
# P — Open market or private purchase of non-derivative or derivative security
# S — Open market or private sale of non-derivative or derivative security
# 6
# V — Transaction voluntarily reported earlier than required
# Rule 16b-3 Transaction Codes
# A — Grant, award or other acquisition pursuant to Rule 16b-3(d)
# D — Disposition to the issuer of issuer equity securities pursuant to Rule 16b-3(e)
# F — Payment of exercise price or tax liability by delivering or withholding securities incident to the receipt, exercise
# or vesting of a security issued in accordance with Rule 16b-3
# I — Discretionary transaction in accordance with Rule 16b-3(f) resulting in acquisition or disposition of issuer
# securities
# M — Exercise or conversion of derivative security exempted pursuant to Rule 16b-3
# Derivative Securities Codes (Except for transactions exempted pursuant to Rule 16b-3)
# C — Conversion of derivative security
# E — Expiration of short derivative position
# H — Expiration (or cancellation) of long derivative position with value received
# O — Exercise of out-of-the-money derivative security
# X — Exercise of in-the-money or at-the-money derivative security
# Other Section 16(b) Exempt Transaction and Small Acquisition Codes (except for Rule 16b-3 codes above)
# G — Bona fide gift
# L — Small acquisition under Rule 16a-6
# W — Acquisition or disposition by will or the laws of descent and distribution
# Z — Deposit into or withdrawal from voting trust
# Other Transaction Codes
# J — Other acquisition or disposition (describe transaction)
# K — Transaction in equity swap or instrument with similar characteristics
# U — Disposition pursuant to a tender of shares in a change of control transaction

# """

# # results.loc[~results['derivativetable_derivativeholding_11_ownershipnature_natureofownership_value'].isna(), ['timestamp', 'issuer_issuertradingsymbol', 'html']]['html'][17]

In [15]:
# feats = [
#     'html',
#     'timestamp',
#     'issuer_issuertradingsymbol',
#     'issuer_issuername',
#     'periodofreport',
#     'reportingowner_reportingownerrelationship_officertitle',
#     'nonderivativetable_nonderivativetransaction_0_securitytitle_value',
#     'nonderivativetable_nonderivativetransaction_0_posttransactionamounts_sharesownedfollowingtransaction_value',
#     'nonderivativetable_nonderivativetransaction_0_transactionamounts_transactionacquireddisposedcode_value',
#     'nonderivativetable_nonderivativetransaction_0_transactionamounts_transactionshares_value',
#     'reportingowner_reportingownerid_rptownername',
#     #> Do they own derivatives? Yes or no.
#     'nonderivativetable_nonderivativetransaction_0_transactioncoding_transactioncode',                                  
#     'nonderivativetable_nonderivativetransaction_0_transactioncoding_transactionformtype',                  
#     'nonderivativetable_nonderivativetransaction_0_transactiondate_value',
# ]

In [16]:
# results[feats].fillna('-').T

In [17]:
# results.loc[0].dropna().keys()

In [18]:
### Check out form 8K

# print('https://sec.gov' + results.loc[36]['html'])